<a href="https://colab.research.google.com/github/hmelino/Notebooks/blob/master/M5_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#M5 Forecasting

This dataset is from Kaggle's competition M5 Forecasting - Accuracy. It consist of Walmart's sales dataset across different shop in 3 states of America. In this notebook we will focus only on first part and that is predicting sales for first 28 days for each item.

In [1]:
import pandas as pd
from fbprophet import Prophet
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool , cpu_count
import pickle

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [2]:
calendar=pd.read_csv('/content/drive/My Drive/Reports/m5-forecasting-accuracy/calendar.csv')
sales_train_val=pd.read_csv('/content/drive/My Drive/Reports/m5-forecasting-accuracy/sales_train_validation.csv')
sample_sub=pd.read_csv('/content/drive/My Drive/Reports/m5-forecasting-accuracy/sample_submission.csv')[:30490]
sell_prices=pd.read_csv('/content/drive/My Drive/Reports/m5-forecasting-accuracy/sell_prices.csv')

Sales_train_val is the main dataset. It contain sale data for individual items in each store. Row for each item have store location, item id, department id and sales for period of 1913 days.

In [3]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,0,3,1,4,1,0,0,3,4,4,0,0,1,0,1,1,7,7,3,6,3,3,7,12,4,2,7,5,12,5,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,5,0,1,1,3,1,1,0,4,2,0,1,2,1,1,0,0,0,0,3,1,1,1,2,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,2,4,1,8,5,2,7,5,3,5,20,8,10,3,3,4,7,2,3,5,6,3,4,1,2,5,1,2,2,2,...,1,1,2,0,0,2,0,0,0,2,0,1,1,0,0,0,0,1,1,2,1,4,0,0,2,2,4,0,0,0,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,2,1,1,2,4,3,0,0,0,0,0,1,1,2,1,1,1,1,2,0,1,0,3,0,0,1,0,0,1,0,3,1,3


#Seasonality
For this competition we have been provided with calendar dataset as well.
We will take information about events and use it to predict more accurate sales.

In [4]:
calendar.head(3)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0


In [5]:
holiday1_dic={holidayName:[date for date in calendar.loc[calendar['event_name_1']==holidayName]['date'].values] for holidayName in calendar['event_name_1'].unique()}

In [6]:
holiday1_dic

{'Chanukah End': ['2011-12-28',
  '2012-12-16',
  '2013-12-05',
  '2014-12-24',
  '2015-12-14'],
 'Christmas': ['2011-12-25',
  '2012-12-25',
  '2013-12-25',
  '2014-12-25',
  '2015-12-25'],
 'Cinco De Mayo': ['2011-05-05',
  '2012-05-05',
  '2014-05-05',
  '2015-05-05',
  '2016-05-05'],
 'ColumbusDay': ['2011-10-10',
  '2012-10-08',
  '2013-10-14',
  '2014-10-13',
  '2015-10-12'],
 'Easter': ['2012-04-08',
  '2013-03-31',
  '2014-04-20',
  '2015-04-05',
  '2016-03-27'],
 'Eid al-Fitr': ['2011-08-31',
  '2012-08-19',
  '2013-08-08',
  '2014-07-29',
  '2015-07-18'],
 'EidAlAdha': ['2011-11-07',
  '2012-10-26',
  '2013-10-15',
  '2014-10-04',
  '2015-09-24'],
 "Father's day": ['2011-06-19', '2012-06-17', '2013-06-16', '2015-06-21'],
 'Halloween': ['2011-10-31',
  '2012-10-31',
  '2013-10-31',
  '2014-10-31',
  '2015-10-31'],
 'IndependenceDay': ['2011-07-04',
  '2012-07-04',
  '2013-07-04',
  '2014-07-04',
  '2015-07-04'],
 'LaborDay': ['2011-09-05',
  '2012-09-03',
  '2013-09-02',
  '20

In [7]:
holiday1_list=[pd.DataFrame({'holiday':hol_name,'ds':pd.to_datetime(dates)}) for hol_name,dates in holiday1_dic.items()]

In [8]:
holiday1_list[1]

,holiday,ds
0,SuperBowl,2011-02-06
1,SuperBowl,2012-02-05
2,SuperBowl,2013-02-03
3,SuperBowl,2014-02-02
4,SuperBowl,2015-02-01
5,SuperBowl,2016-02-07


In [9]:
%%time
holiday = pd.DataFrame(columns=['holiday','ds'])
for d in holiday1_list:
  holiday = pd.concat([holiday,d])


CPU times: user 30.8 ms, sys: 3.49 ms, total: 34.3 ms
Wall time: 31.5 ms


In [10]:
holiday

,holiday,ds
0,SuperBowl,2011-02-06
1,SuperBowl,2012-02-05
2,SuperBowl,2013-02-03
3,SuperBowl,2014-02-02
4,SuperBowl,2015-02-01
...,...,...
0,Easter,2012-04-08
1,Easter,2013-03-31
2,Easter,2014-04-20
3,Easter,2015-04-05


#Main Dataframe

Lets drop all extra columns. All this information is already included in main 'id' feature.

In [11]:
df=sales_train_val.drop(['item_id','dept_id','cat_id','store_id','state_id'],axis=1).set_index(sales_train_val['id']\
                                                                                 .str[:-11])\
                                                                                 .T[1:]\
                                                                                 .set_index(calendar['date'][:1913])

cat_list = list(df.columns)
future = pd.DataFrame({'ds':pd.date_range(start='2016-04-25',end='2016-05-23',freq='d')})
column = 'HOBBIES_1_001_CA_1'

To speed up forecasting , we will only predict necessary 28 days.

In [12]:
future = pd.DataFrame({'ds':pd.date_range(start='2016-04-25',end='2016-05-26')})

#Predict

Because we are dealing with large data, this function was designed to speed up prediction as much as possible. 
- specify seasonality
-uncertainity_samples=0 as we dont need values for yhat lower and yhat upper

Adding holiday will slow this function significantly , adding 3 hours on predicted time, but data will gain great accuracy.

In [13]:
#%%time
#def run_prophet(column):
#  m = Prophet(daily_seasonality=False,weekly_seasonality=False,yearly_seasonality=True,uncertainty_samples=0,holidays=holiday)
#  data = pd.DataFrame({'ds':df[column].index,'y':df[column].values})
#  m.fit(data)
#  return m.predict(future)['yhat'].values
#
#run_prophet(column)

In [14]:
#%%time
#def run_prophet_2(column):
#  m = Prophet(daily_seasonality=False,weekly_seasonality=False,yearly_seasonality=True,uncertainty_samples=0)
#  data = pd.DataFrame({'ds':df[column].index,'y':df[column].values})
#  m.fit(data)
#  return m.predict(future)['yhat'].values
#
#run_prophet(column)



Cell below predicts future sales for 30490 different dataframes. Thats why it took more than 7 hours to complete.

 To avoid the need of running this cell again, I have saved output to pickle file and attached code with its output in this cell for reference. 


```
[IN]
%%time
out=[]

for n in tqdm(cat_list):
  out.append(run_prophet(n))

import pickle
with open('/content/drive/My Drive/Reports/m5-forecasting-accuracy/prediction.pickle','wb') as pickle_file:
  pickle.dump(out,pickle_file)
```



```
[OUT]
100%|██████████| 30490/30490 [7:08:24<00:00,  1.19it/s]CPU times: user 6h 46min 56s, sys: 21min 52s, total: 7h 8min 49s
Wall time: 7h 8min 24s
```



In [15]:
#%%time
#out=[]

#for n in tqdm(cat_list):
#  out.append(run_prophet(n))

In [16]:
import pickle
future_sales=pickle.load(open('/content/drive/My Drive/Reports/m5-forecasting-accuracy/predictions.pickle','rb'))

In [17]:
len(future_sales)

30490

After 7 hours of compute time we have list of predictions for 30490 items , with 28 days of sales predictions each.
It is time to convert it to dataset to perform further calculations.

From data we have created we will 

In [18]:
future_sales[0]

array([0.81087861, 0.80539248, 0.79946431, 0.79306935, 0.78619551,
       0.72358236, 0.60297843, 0.76279178, 0.75416809, 0.74522275,
       0.57968907, 0.72667798, 0.71726285, 0.8248756 , 0.698674  ,
       0.68972776, 0.68116856, 0.67311099, 0.66566503, 0.65893351,
       0.65300963, 0.64797471, 0.64389634, 0.64082676, 0.63880173,
       0.63783981, 0.63794199, 0.63909193])

In [19]:
new_row_df = pd.DataFrame(future_sales[0],index=[f'F{n}' for n in range(1,29)]).T
new_row_df['id']='hey'
new_row_df

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,id
0,0.810879,0.805392,0.799464,0.793069,0.786196,0.723582,0.602978,0.762792,0.754168,0.745223,0.579689,0.726678,0.717263,0.824876,0.698674,0.689728,0.681169,0.673111,0.665665,0.658934,0.65301,0.647975,0.643896,0.640827,0.638802,0.63784,0.637942,0.639092,hey


In [ ]:
%%time
final_df = pd.DataFrame(columns=['id']+[f'F{n}' for n in range(1,29)])
for i in range(len(future_sales)):
  new_row_df = pd.DataFrame(future_sales[i],index=[f'F{n}' for n in range(1,29)]).T
  new_row_df['id']=df.columns[i]
  final_df=pd.concat([final_df,new_row_df])

In [ ]:
final_df.reset_index(drop=True,inplace=True)

In [ ]:
final_df

In [ ]:
plot_df = pd.DataFrame({'y': list(df['HOBBIES_1_001_CA_1'].values.astype('float')) + list(final_df.loc[final_df['id']=='HOBBIES_1_001_CA_1'].values[0][1:]),'day':'None'})
plot_df.loc[plot_df.index < 1913 , 'day'] = 1
plot_df.loc[plot_df.index > 1912 , 'day'] = 2
plot_df

In [ ]:
plt.figure(figsize=[25,7])

plot_df = pd.DataFrame({'y': list(df['HOBBIES_1_008_CA_1'].values.astype('float')) + list(final_df.loc[final_df['id']=='HOBBIES_1_008_CA_1'].values[0][1:])  })
sns.lineplot(data=plot_df,x=plot_df.index,y='y')
plt.axvspan(1912,1940,alpha=0.3,label='prediction')
plt.show()

#Attach prices to predicted unit sales

In [ ]:
sell_prices['code'] = sell_prices['item_id'] +'_'+ sell_prices['store_id']

Lets find mean price values for each item. 

In [ ]:
item_prices = sell_prices.groupby(by='code').mean().drop('wm_yr_wk',axis=1).reset_index().rename(columns={'code':'id'})
item_prices.head(3)

Merge two datasets

In [ ]:
f_df = pd.merge(final_df.sort_values(by='id').round(),item_prices.sort_values(by='id'),on='id')

In [ ]:
f_df.head(3)

In [ ]:
f_df_converted = f_df[f_df.columns[1:-1]]\
       .abs()\
       .apply(lambda x:x*f_df['sell_price'])\

In [ ]:
f_df_converted['id']=f_df['id']

In [ ]:
f_df_converted['state'] = f_df_converted['id'].apply(lambda x: x.split('_')[3])
f_df_converted['cat'] = f_df_converted['id'].apply(lambda x: x.split('_')[0])
f_df_converted['shop_id'] = f_df_converted['id'].apply(lambda x: x.split('_')[4])

After adding our helping columns as state , cat, shop_id we can gain interesting insights from our data.

In [ ]:
sns.barplot(f_df_converted.groupby(by='state').sum().T.sum().astype('float').apply(lambda x: '%.3f' % x))

In [ ]:
f_df_converted.groupby(by='state').sum().T.sum().astype('float').apply(lambda x: '%.3f' % x)

In [ ]:
f_df_converted.groupby(by='state').sum().T.cumsum().astype('float')